#  Modelo de ZeeHB

Sample of notebook for specific model

In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import pandas as pd
import numpy as np
import os, sys, inspect
import commands
import hep as hp

In [3]:
import numpy.lib.scimath as sc

In [4]:
def func(s2phi, M1, M2, MAo, Mho, MHo, v, lam2, lam6, lam7, lam9, lam10, lamh, Muh, Mu2):
    M12 = M1*M1; M22 = M2*M2; MAo2 = MAo*MAo; Mho2 = Mho*Mho; MHo2 = MHo*MHo; Mu22 = Mu2*Mu2; Muh2 = Muh*Muh     
    k = s2phi*np.log(M22/M12)/(4.*np.pi)**2
    phi = 0.5*np.arcsin(s2phi)
    Mu  = (M22-M12)*s2phi/(np.sqrt(2.0)*v)
    MH2 = M12*(np.sin(phi))**2+M22*(np.cos(phi))**2
    M332 = M12*(np.cos(phi))**2+M22*(np.sin(phi))**2
    lam1 = 0.5*(MHo2+Mho2-np.sqrt((MHo2-Mho2)**2-4.0*v**4*lam6**2))/(v**2)
    lam3 = 2.0*(MH2-Mu22)/(v**2)
    lam4 = 0.5*(np.sqrt((MHo2-Mho2)**2-4.0*v**4*lam6**2)+MHo2+Mho2+2.0*(MAo2-2.0*MH2))/(v**2)
    lam5 = 0.5*(np.sqrt((MHo2-Mho2)**2-4.0*v**4*lam6**2)+MHo2+Mho2-2.0*MAo2)/(v**2)
    lam8 = 2.0*(M332-Muh2)/(v**2)
    return k, Mu, lam1, lam3, lam4, lam5, lam8

In [5]:
a=hp.hep(MODEL='radinuZeeHB')

In [6]:
Me   = 0.5109989461e-3     #Masa Electron en GeV
Mmu  = 0.1056583745        #Masa Muon en GeV
Mtau = 1.77686             #Masa Tauon en GeV

S2Sun = .323                                                         #S2Sun: Sino square sun angle 
S2Atm = .573                                                         #S2Atm: Sino square atmospheric angle 
S2Rea = .0229                                                        #S2Rea: Sino square reactor angle 

t12 = np.arcsin(np.sqrt(S2Sun))                               #ThetSun: Sun Angle theta12
t23 = np.arcsin(np.sqrt(S2Atm))                               #ThetAtm: Atmospheric Angle theta23
t13 = np.arcsin(np.sqrt(S2Rea))                               #ThetRea: Reactor Angle theta13

t12 = 33.56*np.pi/180.0 
t13 = 8.46*np.pi/180.0 
t23 = 41.6*np.pi/180.0

f = np.zeros((3,3))
O = np.zeros((3,3))

O[0,0] = 1.0/100000.0
O[0,1] = 1.0/10000.0
O[0,2] = -1.64367e-6
O[1,0] = -0.292587
O[1,1] = 1.0/1000.0
O[1,2] = -0.0000223729
O[2,0] = 1.0/2000.0
O[2,1] = 1.0/10000.0
O[2,2] = 1.0/1000000.0

f[0,1] = 0.00330763
f[0,2] = 0.00838968
f[1,2] = 0.0157395

f[1,0] = -f[0,1]; f[2,0] = -f[0,2]; f[2,1] = -f[1,2]

ft = np.zeros((3,3))
Ot = np.zeros((3,3))

print f
print O
Ot = O
ft = f
O = np.transpose(Ot)
f = np.transpose(ft)

[[ 0.          0.00330763  0.00838968]
 [-0.00330763  0.          0.0157395 ]
 [-0.00838968 -0.0157395   0.        ]]
[[  1.00000000e-05   1.00000000e-04  -1.64367000e-06]
 [ -2.92587000e-01   1.00000000e-03  -2.23729000e-05]
 [  5.00000000e-04   1.00000000e-04   1.00000000e-06]]


In [7]:
v = a.vev
s2phi = 1.4E-02
M1 = 200.0
M2 = 300.0
MAo = 90.0
Mho = 125.0
MHo = 700.0
lam2 = 1.3000000E-01    # lambda2Input
lam6 = 1.000000E-03     # lambda6Inputd 
lam7 = 0.000000E+00     # lambda7Input
lam9 = 0.000000E+00     # lambda9Input
lam10 = 0.000000E+00    # lambda10Input
lamh = 0.000000E+00     # lambdahInput
Muh = 1.000000E+02      # MhInput
Mu2 = 2.0000000E+02     # mEt2Input

k, Mu, lam1, lam3, lam4, lam5, lam8 = func(s2phi, M1, M2, MAo, Mho, MHo, v, lam2, lam6, lam7, lam9, lam10, lamh, Muh, Mu2)
print k
devnull=commands.getoutput('rm -f SPheno.spc.%s' %a.MODEL)

a.LHA.blocks['SPHENOINPUT'].entries[55]='0               # Calculate one loop masses'
a.LHA.blocks['MINPAR'][1]='%0.8E       #lambda1Input' %lam1
a.LHA.blocks['MINPAR'][2]='%0.8E       #lambda2Input' %lam2
a.LHA.blocks['MINPAR'][3]='%0.8E       #lambda3Input' %lam3
a.LHA.blocks['MINPAR'][4]='%0.8E       #lambda4Input' %lam4
a.LHA.blocks['MINPAR'][5]='%0.8E       #lambda5Input' %lam5
a.LHA.blocks['MINPAR'][6]='%0.8E       #lambda6Input' %lam6
a.LHA.blocks['MINPAR'][7]='%0.8E       #lambda7Input' %lam7
a.LHA.blocks['MINPAR'][8]='%0.8E       #lambda8Input' %lam8
a.LHA.blocks['MINPAR'][9]='%0.8E       #lambda9Input' %lam9
a.LHA.blocks['MINPAR'][10]='%0.8E      #lambda10Input'%lam10
a.LHA.blocks['MINPAR'][11]='%0.8E      #lambdahInput' %lamh
a.LHA.blocks['MINPAR'][12]='%0.8E      #MhInput'      %(Muh*Muh)
a.LHA.blocks['MINPAR'][13]='%0.8E      #MuInput'      %Mu
a.LHA.blocks['MINPAR'][14]='%0.8E      #mEt2Input'    %(Mu2*Mu2)

7.18938581886e-05


In [8]:
pd.Series(a.LHA.blocks['MINPAR'].entries)

1     2.57733920E-01       #lambda1Input
2     1.30000000E-01       #lambda2Input
3     1.64941544E+00       #lambda3Input
4     5.24712835E+00       #lambda4Input
5     7.94892241E+00       #lambda5Input
6     1.00000000E-03       #lambda6Input
7     0.00000000E+00       #lambda7Input
8     9.89778593E-01       #lambda8Input
9     0.00000000E+00       #lambda9Input
10    0.00000000E+00      #lambda10Input
11     0.00000000E+00      #lambdahInput
12          1.00000000E+04      #MhInput
13          2.01029000E+00      #MuInput
14        4.00000000E+04      #mEt2Input
dtype: object

In [9]:
mlep=np.diag([Me,Mmu,Mtau])
m1= 0.0  # lightest neutrino
Yh12 = f[0,1]     # Yh(1,2)
Yh13 = f[0,2]     # Yh(1,3)
Yh21 = f[1,0]     # Yh(2,1)
Yh23 = f[1,2]     # Yh(2,3)
Yh31 = f[2,0]     # Yh(3,1)
Yh32 = f[2,1]     # Yh(3,2)

a.LHA.blocks['YHIN'][(1,2)]='%0.8E      # Yh(1,2)'    %Yh12
a.LHA.blocks['YHIN'][(1,3)]='%0.8E      # Yh(1,3)'    %Yh13
a.LHA.blocks['YHIN'][(2,1)]='%0.8E      # Yh(2,1)'    %Yh21
a.LHA.blocks['YHIN'][(2,3)]='%0.8E      # Yh(2,3)'    %Yh23
a.LHA.blocks['YHIN'][(3,1)]='%0.8E      # Yh(3,1)'    %Yh31
a.LHA.blocks['YHIN'][(3,2)]='%0.8E      # Yh(3,2)'    %Yh32

epsE11 = O[0,0]          # epsE(1,1)
epsE12 = O[0,1]          # epsE(1,2)
epsE13 = O[0,2]          # epsE(1,3)
epsE21 = O[1,0]          # epsE(2,1)
epsE22 = O[1,1]          # epsE(2,2)
epsE23 = O[1,2]          # epsE(2,3)
epsE31 = O[2,0]          # epsE(3,1)
epsE32 = O[2,1]          # epsE(3,2)
epsE33 = O[2,2]          # epsE(3,3)

a.LHA.blocks['EPSEIN'][(1,1)]='%0.8E      # epsE(1,1)'    %epsE11
a.LHA.blocks['EPSEIN'][(1,2)]='%0.8E      # epsE(1,2)'    %epsE12
a.LHA.blocks['EPSEIN'][(1,3)]='%0.8E      # epsE(1,3)'    %epsE13
a.LHA.blocks['EPSEIN'][(2,1)]='%0.8E      # epsE(2,1)'    %epsE21
a.LHA.blocks['EPSEIN'][(2,2)]='%0.8E      # epsE(2,2)'    %epsE22
a.LHA.blocks['EPSEIN'][(2,3)]='%0.8E      # epsE(2,3)'    %epsE23
a.LHA.blocks['EPSEIN'][(3,1)]='%0.8E      # epsE(3,1)'    %epsE31
a.LHA.blocks['EPSEIN'][(3,2)]='%0.8E      # epsE(3,2)'    %epsE32
a.LHA.blocks['EPSEIN'][(3,3)]='%0.8E      # epsE(3,3)'    %epsE33

moc=a.runSPheno()

In [10]:
a.LHA_out_with_comments.blocks['MASS'].entries

_dict([(25, '1.25000000E+02  # hh_1'),
       (35, '7.00000000E+02  # hh_2'),
       (37, '2.00000000E+02  # Hm_2'),
       (900037, '3.00000000E+02  # Hm_3'),
       (36, '9.00000003E+01  # etI'),
       (23, '9.11887000E+01  # VZ'),
       (24, '8.03497269E+01  # VWp'),
       (1, '5.00000000E-03  # Fd_1'),
       (3, '9.50000000E-02  # Fd_2'),
       (5, '4.18000000E+00  # Fd_3'),
       (2, '2.50000000E-03  # Fu_1'),
       (4, '1.27000000E+00  # Fu_2'),
       (6, '1.73500000E+02  # Fu_3'),
       (11, '5.10998930E-04  # Fe_1'),
       (13, '1.05658372E-01  # Fe_2'),
       (15, '1.77669000E+00  # Fe_3'),
       (12, '1.10566570E-15  # Fv_1'),
       (14, '8.79913008E-12  # Fv_2'),
       (16, '-5.08788553E-11 # Fv_3')])

In [11]:
m_e=0.5109989461e-3;m_mu=0.1056583745;m_tau=1.77686
mlep=np.diag([m_e,m_mu,m_tau])
Mnu = k*( np.dot( np.dot(O,mlep),f.transpose() )+np.dot( np.dot(f,mlep),O.transpose()) )
print Mnu
Mnu_diag,U=np.linalg.eig(Mnu)
#{-1.5019*10^-12, 2.15866*10^-12, 8.37483*10^-12}, 
#{2.15866*10^-12, 1.88608*10^-11, 2.74299*10^-11}, 
#{8.37483*10^-12, 2.74299*10^-11, 2.42203*10^-11}

[[  1.50189445e-12  -2.15866086e-12  -8.37482222e-12]
 [ -2.15866086e-12  -1.88608035e-11  -2.74299015e-11]
 [ -8.37482222e-12  -2.74299015e-11  -2.42202582e-11]]


In [12]:
lo=np.argsort(np.abs(Mnu_diag))
Mnu_diag=np.array([Mnu_diag[lo[0]],Mnu_diag[lo[1]],Mnu_diag[lo[2]]])
U=np.matrix(U)
U=np.asarray(np.hstack((U[:,lo[0]],U[:,lo[1]],U[:,lo[2]])))

In [13]:
Mnu_diag # m

array([  4.78775911e-18,   8.66025815e-12,  -5.02394302e-11])

In [14]:
U

array([[ 0.82424074, -0.54679359,  0.1471189 ],
       [-0.49478359, -0.56914989,  0.65670206],
       [ 0.27534777,  0.61407261,  0.73966103]])

In [15]:
mltmp,Dm21_2,Dm3l_2,theta12,theta23,theta13,delta=hp.neutrino_data(IH=False)
#m2=np.sqrt(Dm21_2[1]+m1**2); m3=np.sqrt(Dm3l_2[1]+m1**2)
np.abs( hp.UPMNS(theta12[1],theta13[1],theta23[1]) )

array([[ 0.82424   ,  0.5467947 ,  0.14711891],
       [ 0.49478442,  0.56914935,  0.65670191],
       [ 0.27534849,  0.61407213,  0.73966116]])

In [16]:
print O
print f

[[  1.00000000e-05   1.00000000e-04  -1.64367000e-06]
 [ -2.92587000e-01   1.00000000e-03  -2.23729000e-05]
 [  5.00000000e-04   1.00000000e-04   1.00000000e-06]]
[[ 0.          0.00330763  0.00838968]
 [-0.00330763  0.          0.0157395 ]
 [-0.00838968 -0.0157395   0.        ]]


[![Home](http://www.incredimail.com/images/nav%20bar/home-icon.png)](./) 
[Jupyter home](./draft.pdf)